In [ ]:
Prepared by Charles Kangai, ROI Training Inc., email: charles@charleskangai.co.uk

Examples to be used with the tables created by bash script: import_nwind.sh. The script creates a Bigquery dataset named demos and creates tables in the dataset.
    Upload the files customers_tab.csv, employees_tab.csv, order_details_tab.csv, products_tab.csv, regions_tab.csv and suppliers_tab.csv to Google Cloud Shell 
    then execute the import_nwind.sh script supplying the project_id as a command ine argument, e.g.:
    ./import_nwind.sh qwiklabs-gcp-04-5bb209450a77
    
Disclaimer: these scripts have been prepared for use in class only for instructional purposes by Charles Kangai and students and for no other purpose.

In [ ]:
Other aggregate functions are: ANY_VALUE, ARRAY_AGG, ARRAY_CONCAT_AGG, AVG, BIT_AND, BIT_OR, BIT_XOR, COUNT, LOGICAL_AND, LOGICAL_OR, MAX, MIN, SUM, STRING_AGG, CORR, COVAR_POP, COVAR_SAMP
    , STDDEV_POP, STDDEV_SAMP, VAR_POP, VAR_SAMP

In [ ]:
%%bigquery
SELECT 
COUNTIF(product_name = 'Chai') AS ChaiSales 
FROM demos.order_details 

Illustrate using ARRAY_AGG and Bigquery concatenation of strings with operator

In [ ]:
%%bigquery
SELECT product_name, 
       ARRAY_AGG(c.company_name)                         AS buyers, 
       ARRAY_AGG(DISTINCT 'Shipper ' 
                          || Cast(o.ship_via AS STRING)) AS shippers 
FROM   demos.order_details AS od 
       JOIN demos.orders AS o using(order_id) 
       JOIN demos.customers AS c using(customer_id) 
GROUP  BY od.product_name 

In [ ]:
STRING_AGG function - this is supported in many relational databases, e.g. SQL Server

In [ ]:
%%bigquery
SELECT customer_id, 
       STRING_AGG(CAST(order_id AS STRING)) AS orders 
FROM   demos.orders 
GROUP  BY customer_id 

Statistical correlation between two variables, based on the Spearman formula

In [ ]:
%%bigquery
SELECT CORR(quantity, unit_price) AS correlation 
FROM demos.order_details

Counting distinct values

In [ ]:
%%bigquery
SELECT
  COUNT(DISTINCT product_name) AS products_sold
FROM
  demos.order_details

COUNT(DISTINCT) is very expensive. Consider using APPROX_COUNT_DISTINCT

In [ ]:
%%bigquery
SELECT
  APPROX_COUNT_DISTINCT(product_name) AS approx_products_sold
FROM
  demos.order_details

Quantiles are a stepping stone to finding percentiles like lower quartile, median, upper quartile, 90th percentile, etc.

In [ ]:
%%bigquery
SELECT
  APPROX_QUANTILES(x, 2) AS approx_quantiles
FROM
  UNNEST([1, 1, 1, 4, 5, 6, 7, 8, 9, 10]) AS x;

In [ ]:
%%bigquery
SELECT
  APPROX_QUANTILES(x, 2)[OFFSET(1)] AS median
FROM
  UNNEST([1, 1, 1, 4, 5, 6, 7, 8, 9, 10]) AS x;

In [ ]:
%%bigquery
SELECT APPROX_QUANTILES(net_amount, 2) AS approx_quantiles 
FROM   (SELECT order_id, 
               Sum(unit_price * quantity * ( 1 - discount )) AS net_amount 
        FROM   demos.order_details 
        GROUP  BY order_id) 

Finding top counts

In [ ]:
%%bigquery
WITH product_sales 
     AS (SELECT product_name 
         FROM   demos.order_details) 
SELECT APPROX_TOP_COUNT(product_name, 5) AS Top_5_occurring
FROM   product_sales

Split a range into buckets

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE demos.t 
  ( 
     val INT64 
  ); 

INSERT INTO demos.t 
            (val) 
VALUES     (1), 
            (2), 
            (3), 
            (4), 
            (5), 
            (6), 
            (7), 
            (8), 
            (9); 

SELECT val, 
       Ntile(4) 
         OVER( 
           ORDER BY val) AS bucket_number 
FROM   demos.t 

Find highest or lowest value across columns

In [ ]:
%%bigquery
SELECT units_in_stock, 
       units_on_order, 
       reorder_level, 
       GREATEST(units_in_stock, units_on_order, reorder_level) AS highest 
FROM   demos.products 

Compare values across rows by using navigation functions: LAG, LEAD, NTH_VALUE, etc.

In [ ]:
%%bigquery
SELECT o.order_date,
       SUM(unit_price * quantity)                              AS order_amount,
       SUM(unit_price * quantity) - LAG(SUM(unit_price * quantity))
                                      OVER(
                                        ORDER BY o.order_date) AS order_increase
FROM   demos.order_details AS od
       JOIN demos.orders AS o using(order_id)
GROUP  BY o.order_date
ORDER  BY o.order_date 

Find percentiles

In [ ]:
%%bigquery
SELECT
  PERCENTILE_CONT(h, 0.5) OVER() AS median
FROM
  UNNEST(GENERATE_ARRAY(0, 10, 1)) AS h
LIMIT 1

In [ ]:
%%bigquery
WITH order_amounts
     AS (SELECT order_id,
                SUM(unit_price * quantity) AS order_amount
         FROM   demos.order_details
         GROUP  BY order_id)
SELECT PERCENTILE_CONT(order_amount, 0.5)
         OVER() AS median_order_amount
FROM   order_amounts 
LIMIT 1;

Hashing functions: FARM_FINGERPRINT, MD5, SHA1, SHA256, SHA512. These last output BYTES rather than number.

In [ ]:
%%bigquery
SELECT *
FROM   demos.orders
WHERE  ABS(MOD(Farm_fingerprint(freight
                            || order_id
                            || employee_id), 4)) = 1 

Regular expressions - very important: REGEXP_CONTAINS, REGEXP_EXTRACT, REGEXP_EXTRACT_ALL. Based on re2 standard.
Other important string functions are STRPOS, SUBSTR, REPLACE, LOWER, UPPER, ... Not explicitly covered here as it is assumed they are common enough.

In [ ]:
%%bigquery
WITH
  emails AS (
  SELECT
    "foo@example.com" AS email
  UNION ALL
  SELECT
    "bar@example.org"
  UNION ALL
  SELECT
    "www.example.net"
  UNION ALL
  SELECT
    "hello@domain.some!" )
SELECT
  email,
  REGEXP_CONTAINS(email, r"@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+") AS is_valid
FROM
  emails

In [ ]:
%%bigquery
WITH
  emails AS (
  SELECT
    "foo@example.com" AS email
  UNION ALL
  SELECT
    "bar@example.org"
  UNION ALL
  SELECT
    "www.example.net"
  UNION ALL
  SELECT
    "hello@domain.some!" )
SELECT
  email,
  REGEXP_CONTAINS(email, r"@[a-zA-Z0-9-]+\.[a-zA-Z0-9]+") AS is_valid
FROM emails

In [ ]:
%%bigquery
WITH
  emails AS (
  SELECT
    "foo@example.com" AS email
  UNION ALL
  SELECT
    "bar@example.org"
  UNION ALL
  SELECT
    "www.example.net"
  UNION ALL
  SELECT
    "hello@domain.some!" )
SELECT
  email,REGEXP_CONTAINS(email, r"@[a-zA-Z0-9-]+\.[a-zA-Z0-9$]+") AS is_valid
FROM emails;

In [ ]:
%%bigquery
WITH email_addresses AS
  (SELECT "charles@roitraining.com" as email
  UNION ALL
  SELECT "dave@royalacademy.org" as email
  UNION ALL
  SELECT "charles_kangai@roitraining.com" as email
  UNION ALL 
  SELECT "charles.kangai@roitraining.com" 
  UNION ALL 
  SELECT "charles+kangai@roitraining.com"
  UNION ALL 
  SELECT "john-doe@example.com")

SELECT
  REGEXP_EXTRACT(email, r"^[a-zA-Z0-9_.+-]+")
  AS user_name
FROM email_addresses;

In [ ]:
%%bigquery
WITH email_addresses AS
  (SELECT "foo@example.com" as email
  UNION ALL
  SELECT "bar@example.org" as email
  UNION ALL
  SELECT "baz@example.net" as email)

SELECT email, 
  REGEXP_EXTRACT(email, r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.([a-zA-Z0-9-.]+$)")
  AS top_level_domain
FROM email_addresses;

In [ ]:
%%bigquery
WITH code_markdown AS
  (SELECT "Try `function(x)` or `function(y)`" as code)
SELECT
  REGEXP_EXTRACT_ALL(code, "`(.+?)`") AS example
FROM code_markdown;

In [ ]:
Generating a sequence.

In [ ]:
%%bigquery
SELECT GENERATE_ARRAY(1, 100) AS One_to_100;

In [ ]:
%%bigquery
SELECT GENERATE_DATE_ARRAY(DATE '2020-06-01', DATE '2020-06-30') AS June_Days;

Date and Time functions - really important.

In [ ]:
%%bigquery
SELECT 	CURRENT_DATE() AS the_date
		,CURRENT_DATETIME() AS date_and_time
		,CURRENT_TIMESTAMP() AS date_and_time_with_zone

In [ ]:
%%bigquery
SELECT 	DATE_ADD(CURRENT_DATE(), INTERVAL 2 WEEK) AS Two_Weeks_Hence
		    ,CURRENT_DATETIME()                                 AS date_time_now
		    ,DATETIME_ADD(CURRENT_DATETIME(), INTERVAL 36 hour) AS Thirty_6_hrs_later 
		    ,TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 HOUR) AS one_hour_ago;

In [ ]:
%%bigquery
SELECT 	DATE_DIFF(DATE '2020-12-25', CURRENT_DATE(), DAY) AS Days_to_Xmas
		,DATETIME_DIFF(DATETIME '2020-06-16 10:00:00', CURRENT_DATETIME(), HOUR) AS hours_between
		,DATETIME_SUB(DATETIME '2020-06-03 10:00:00', INTERVAL 25 HOUR) AS yest_minus_hour;

In [ ]:
%%bigquery
SELECT DATE_TRUNC(CURRENT_DATE(), MONTH) AS beginning_of_month, DATE_TRUNC(CURRENT_DATE(), YEAR) AS beginning_of_year

In [ ]:
%%bigquery
SELECT FORMAT_DATE('%d-%m-%Y', CURRENT_DATE()) AS UK_Date

Other date formats available here: https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators#timezone_definitions

In [ ]:
%%bigquery
SELECT 	
		FORMAT_DATE('%d-%b-%Y', CURRENT_DATE()) AS abbr_month
		,FORMAT_DATE('%d-%B-%Y', CURRENT_DATE()) AS full_month
		,FORMAT_DATE('%a', CURRENT_DATE()) AS short_week_day_name 
		,FORMAT_DATE('%A', CURRENT_DATE()) AS long_week_day_name

In [ ]:
%%bigquery
SELECT PARSE_DATE('%d-%m-%Y', '25-12-2020') AS from_uk_format

In [ ]:
%%bigquery
SELECT UNIX_DATE(CURRENT_DATE()) AS days_since_jan_1_1970

In [ ]:
%%bigquery
SELECT EXTRACT(DAYOFWEEK FROM CURRENT_TIMESTAMP()) AS day_number_of_week
		,FORMAT_TIMESTAMP('%A', CURRENT_TIMESTAMP()) AS day_name_of_week;

In [ ]:
%%bigquery
SELECT TIMESTAMP_SECONDS(36000000) as thirty_six_million_sec_after_epoch_time

In [ ]:
%%bigquery
SELECT TIMESTAMP_MILLIS(36000000000) AS thirty_six_million_sec_after_epoch_time

In [ ]:
%%bigquery
SELECT UNIX_MILLIS(CURRENT_TIMESTAMP()) AS milliseconds_passed;

Error function - not like RAISERROR or RAISE_APPLICATION_ERROR as this can be used inline with SQL statements

In [ ]:
%%bigquery
SELECT
  CASE
    WHEN grade = 1 THEN 'Distinction'
    WHEN grade = 2 THEN 'Passed'
    WHEN grade = 3 THEN 'Remediation'
    ELSE ERROR(concat('Not a standard grade: ', grade))
  END AS new_value
FROM (
  SELECT 1 AS grade UNION ALL
  SELECT 2   UNION ALL
  SELECT 3   UNION ALL 
  SELECT 4);

In [ ]:
%%bigquery
WITH groceries AS 
(SELECT 1 as id, ['apple','pear','banana'] AS list 
UNION ALL SELECT 2, ['carrot','apple'] AS list 
UNION ALL SELECT 3, ['water','wine'] AS list)
SELECT id, list 
  FROM groceries
  WHERE 'apple' IN UNNEST(list)

Denormalize

In [ ]:
%%bigquery
WITH cte_od 
     AS (SELECT o.order_id, 
                o.customer_id, 
                o.order_date, 
                o.shipped_date, 
                o.freight, 
                ARRAY_AGG(STRUCT (od.order_id, od.product_id, od.unit_price, 
                          od.quantity, 
                          od.discount, 
                                    od.product_name)) AS line_items 
         FROM   demos.order_details AS od 
                JOIN demos.orders AS o 
                  ON o.order_id = od.order_id 
         GROUP  BY o.order_id, 
                   o.customer_id, 
                   o.order_date, 
                   o.shipped_date, 
                   o.freight) 
SELECT c.customer_id, 
       c.company_name, 
       c.contact_name, 
       ARRAY_AGG(STRUCT(co.order_id, co.customer_id, co.order_date, 
                 co.shipped_date, 
                 co.freight, 
                           co.line_items)) AS customer_orders 
FROM   demos.customers AS c 
       JOIN cte_od AS co 
         ON c.customer_id = co.customer_id 
GROUP  BY c.customer_id, 
          c.company_name, 
          c.contact_name 


create the denormalized table

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE demos.customer_and_orders AS 
WITH cte_od 
     AS (SELECT o.order_id, 
                o.customer_id, 
                o.order_date, 
                o.shipped_date, 
                o.freight, 
                ARRAY_AGG(STRUCT (od.order_id, od.product_id, od.unit_price, 
                          od.quantity, 
                          od.discount, 
                                    od.product_name)) AS line_items 
         FROM   demos.order_details AS od 
                JOIN demos.orders AS o 
                  ON o.order_id = od.order_id 
         GROUP  BY o.order_id, 
                   o.customer_id, 
                   o.order_date, 
                   o.shipped_date, 
                   o.freight) 
SELECT c.customer_id, 
       c.company_name, 
       c.contact_name, 
       ARRAY_AGG(STRUCT(co.order_id, co.customer_id, co.order_date, 
                 co.shipped_date, 
                 co.freight, 
                           co.line_items)) AS customer_orders 
FROM   demos.customers AS c 
       JOIN cte_od AS co 
         ON c.customer_id = co.customer_id 
GROUP  BY c.customer_id, 
          c.company_name, 
          c.contact_name 

List customers with more than 10 orders

In [ ]:
%%bigquery
SELECT * 
FROM demos.customer_and_orders 
WHERE ARRAY_LENGTH(customer_orders) > 10

List customers who bought Longlife Tofu, along with the date of the order and the order number

In [ ]:
%%bigquery
SELECT
  cto.customer_id
  ,company_name
  ,co.order_id
  ,co.order_date
  ,li.product_name
FROM
  demos.customer_and_orders AS cto,
  UNNEST(customer_orders) AS co,
  UNNEST(co.line_items) AS li
WHERE
  li.product_name = 'Longlife Tofu'

Generate a UUID:

In [62]:
%%bigquery
SELECT GENERATE_UUID() AS Globally_Unique;

,Globally_Unique
0,b3345b1f-66e9-4c13-9359-31c0071324bb


Extract an element from an array

In [ ]:
%%bigquery
SELECT SPLIT('10251,22,16.8,6,5.0000001E-2,Gustaf\'s Knäckebröd', ',')[OFFSET(5)] AS product_name;

In [ ]:
%%bigquery
WITH
  order_lines AS (
  SELECT
    '10248,11,14,12,0,Queso Cabrales' AS order_info
  UNION ALL
  SELECT
    '10249,51,42.4,40,0,Manjimup Dried Apples'
  UNION ALL
  SELECT
    '10250,65,16.8,15,0.15000001,Louisiana Fiery Hot Pepper Sauce'
  UNION ALL
  SELECT
    '10252,20,64.8,40,5.0000001E-2,Sir Rodney\'s Marmalade' )
SELECT
  SPLIT(order_info, ',')[OFFSET(0)] AS order_id,
  SPLIT(order_info, ',')[OFFSET(5)] AS product_name
FROM
  order_lines

Extract from JSON

In [ ]:
%%bigquery
SELECT JSON_EXTRACT(json_string, '$')
FROM UNNEST(['{   "city": "Vancouver",   "country": "Canada",   "region": "British Columbia"  }'
,  '{    "city": "Tsawassen",    "country": "Canada",    "region": "British Columbia"  }' 
,  '{    "city": "Montréal",    "country": "Canada",    "region": "Québec"  }'])
AS json_string

In [ ]:
%%bigquery
SELECT 
    JSON_EXTRACT(json_string, '$.city') AS city
    ,JSON_EXTRACT(json_string, '$.country') AS country
    ,JSON_EXTRACT(json_string, '$.region') AS region
FROM UNNEST(['{   "city": "Vancouver",   "country": "Canada",   "region": "British Columbia"  }'
,  '{    "city": "Tsawassen",    "country": "Canada",    "region": "British Columbia"  }' 
,  '{    "city": "Montréal",    "country": "Canada",    "region": "Québec"  }'])
AS json_string

Return JSON with TO_JSON_STRING

In [ ]:
%%bigquery
with orders AS (
SELECT order_id, order_date, ARRAY( SELECT STRUCT( product_id, unit_price, quantity)FROM demos.order_details WHERE order_id = o.order_id) AS details 
FROM demos.orders AS o 
LIMIT 5)
SELECT t, TO_JSON_STRING(t)
FROM orders AS t